# Reducing science images


In [ ]:
from pathlib import Path

from matplotlib import pyplot as plt
import numpy as np

from astropy import units as u
import ccdproc as ccdp

from convenience_functions import show_image

## Overview

[ccdproc](ccdproc.readthedocs.io) provides a couple of ways to approach calibration of the science images:

+ Perform each of the individual steps manually using `subtract_bias`, `subtract_dark`, and `flat_correct`.
+ Use the [`ccd_process` function]() to perform all of the reduction steps. 

This notebook will do each of those in separate sections below.

In [ ]:
def find_nearest_dark_exposure(image, dark_exposure_times, tolerance=0.5):
    """
    Find the nearest exposure time of a dark frame to the exposure time of the image,
    raising an error if the difference in exposure time is more than tolerance.
    
    Parameters
    ----------
    
    image : astropy.nddata.CCDData
        Image for which a matching dark is needed.
    
    dark_exposure_times : list
        Exposure times for which there are darks.
    
    tolerance : float or ``None``, optional
        Maximum difference, in seconds, between the image and the closest dark. Set
        to ``None`` to skip the tolerance test.
    
    Returns
    -------
    
    float
        Closest dark exposure time to the image.
    """

    dark_exposures = np.array(list(dark_exposure_times))
    idx = np.argmin(np.abs(dark_exposures - image.header['exptime']))
    closest_dark_exposure = dark_exposures[idx]

    if (tolerance is not None and 
        np.abs(image.header['exptime'] - closest_dark_exposure) > tolerance):
        
        raise RuntimeError('Closest dark exposure time is {} for flat of exposure '
                           'time {}.'.format(closest_dark_exposure, a_flat.header['exptime']))
        
    
    return closest_dark_exposure

## Example 1


In [ ]:
reduced_path = Path('example1-reduced')

science_imagetyp = 'object'
flat_imagetyp = 'flatfield'

ifc_reduced = ccdp.ImageFileCollection('example1-reduced')
ifc_raw = ccdp.ImageFileCollection('python_imred_data')

In [ ]:
lights = ifc_raw.summary[ifc_raw.summary['imagetyp'] == science_imagetyp.upper()]
lights['date-obs', 'file', 'object', 'filter', 'exptime']

In [ ]:
combined_darks = {ccd.header['exptime']: ccd for ccd in ifc_reduced.ccds(imagetyp='dark', combined=True)}

In [ ]:
calibs = {}
for im_type in ['bias', 'dark', flat_imagetyp]:
    calibs[im_type] = [ccd for ccd in ifc_reduced.ccds(imagetyp=im_type, combined=True)]

# THIS IS BAD NEED TO FIND MATCHING DARK EXPOSURE TIME!! 
# AND MATCHING FLAT FILTER!!!!

In [ ]:
all_reds = []
light_ccds = []
for light, file_name in ifc_raw.ccds(imagetyp=science_imagetyp, return_fname=True, ccd_kwargs=dict(unit='adu')):
    light_ccds.append(light)
    reduced = ccdp.subtract_overscan(light, overscan=light[:, 2055:], median=True)
    reduced = ccdp.trim_image(light[:, :2048])
    #reduced = ccdp.subtract_bias(reduced, calibs['bias'][0])
    closest_dark = find_nearest_dark_exposure(reduced, combined_darks.keys())
    reduced = ccdp.subtract_dark(reduced, combined_darks[closest_dark], exposure_time='exptime', exposure_unit=u.second)
    good_flat = [c for c in calibs[flat_imagetyp] if c.header['filter'] == light.header['filter']][0]
    reduced = ccdp.flat_correct(reduced, good_flat)
    all_reds.append(reduced)
    reduced.write(reduced_path / file_name)

In [ ]:
which_light = 1
fig, axes = plt.subplots(1, 2, figsize=(20, 10))
filt = light_ccds[which_light].header['filter']
axes[0].set_title('Uncalibrated image {}'.format(filt))
show_image(light_ccds[which_light], cmap='gray', ax=axes[0], fig=fig, percl=90)

axes[1].set_title('Calibrated image')
show_image(all_reds[which_light].data, cmap='gray', ax=axes[1], fig=fig, percl=90)

In [ ]:
show_image(combined_darks[300], cmap='gray', percl=80)
show_image(calibs['bias'][0], cmap='gray', percl=80)
show_image(calibs[flat_imagetyp][0], cmap='gray', percl=90)

## Example 2

In [ ]:
reduced_path = Path('example2-reduced')

science_imagetyp = 'light'

ifc_reduced = ccdp.ImageFileCollection('example2-reduced')
ifc_raw = ccdp.ImageFileCollection('example-thermo-electric')

Grab the light images...

In [ ]:
lights = ifc_raw.summary[ifc_raw.summary['imagetyp'] == science_imagetyp.upper()]
lights['date-obs', 'file', 'object', 'filter', 'exposure']

Grab some calibration frames

In [ ]:
calibs = {}
for im_type in ['bias', 'dark', 'flat']:
    calibs[im_type] = [ccd for ccd in ifc_reduced.ccds(imagetyp=im_type, combined=True)]

Let's reduce them all...

# BUT FIX THE FACT THAT WE ARE NOT MATCHING WE ARE JUST GRABBING THE FIRST ONE!

In [ ]:
all_reds = []
light_ccds = []
for light, file_name in ifc_raw.ccds(imagetyp='light', return_fname=True):
    light_ccds.append(light)
    reduced = ccdp.trim_image(light[:, :4096])
    reduced = ccdp.subtract_bias(reduced, calibs['bias'][0])
    reduced = ccdp.subtract_dark(reduced, calibs['dark'][0], exposure_time='exptime', exposure_unit=u.second)
    reduced = ccdp.flat_correct(reduced, calibs['flat'][0])
    all_reds.append(reduced)
    reduced.write(reduced_path / file_name)

In [ ]:
which_light = 0
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

axes[0].set_title('Uncalibrated image')
show_image(light_ccds[which_light], cmap='gray', ax=axes[0], fig=fig, percl=90)

axes[1].set_title('Calibrated image')
show_image(all_reds[which_light].data, cmap='gray', ax=axes[1], fig=fig, percl=90)

In [ ]:
which_light = 1
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

axes[0].set_title('Uncalibrated image')
show_image(light_ccds[which_light], cmap='gray', ax=axes[0], fig=fig, percl=90)

axes[1].set_title('Calibrated image')
show_image(all_reds[which_light].data, cmap='gray', ax=axes[1], fig=fig, percl=90)